In [15]:
# https://github.com/kserve/kserve/tree/master/docs

### iris-example.yaml
```
apiVersion: "serving.kserve.io/v1beta1"
kind: "InferenceService"
metadata:
  annotations:
    sidecar.istio.io/inject: "false"
  name: "sklearn-iris"
spec:
  predictor:
    sklearn:
      image: "kserve/sklearnserver:v0.9.0"
      storageUri: "gs://kfserving-examples/models/sklearn/1.0/model"
```
- example의 model은 kserve8.0 이상에서 동작하도록 만들어져 있기 때문에 image를 따로 불러와야 한다. 

### iris-request.yaml
```
{
    "instances": [
      [6.8,  2.8,  4.8,  1.4],
      [6.0,  3.4,  4.5,  1.6]
    ]
}
```

In [16]:
!pip install kserve==0.8

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfserving 0.5.1 requires azure-storage-blob<=2.1.0,>=1.3.0, but you have azure-storage-blob 12.8.1 which is incompatible.


In [17]:
import warnings
warnings.filterwarnings("ignore")

In [18]:
from kserve import (
    constants,
    KServeClient,
    V1beta1InferenceService,
    V1beta1InferenceServiceSpec,
    V1beta1PredictorSpec,
    V1beta1SKLearnSpec,
    V1beta1ModelSpec,
    V1beta1ModelFormat,
    V1beta1TFServingSpec,
    utils
)
from kubernetes import client 

In [19]:
namespace = utils.get_default_target_namespace()
service_name="sklearn-iris"

api_version = constants.KSERVE_GROUP + '/' + constants.KSERVE_V1BETA1_VERSION
storage = 'gs://kfserving-examples/models/sklearn/1.0/model'
image = "kserve/sklearnserver:v0.9.0"
isvc = V1beta1InferenceService(api_version=api_version,
                               kind=constants.KSERVE_KIND,
                               metadata=client.V1ObjectMeta(name=service_name, namespace=namespace, annotations={'sidecar.istio.io/inject':'false'}),
                               spec=V1beta1InferenceServiceSpec(
                                   predictor=V1beta1PredictorSpec(
                                       sklearn=(V1beta1SKLearnSpec(
                                           image = image,
                                            storage_uri = storage))))
)

In [20]:
KServe = KServeClient()
KServe.create(isvc)

{'apiVersion': 'serving.kserve.io/v1beta1',
 'kind': 'InferenceService',
 'metadata': {'annotations': {'sidecar.istio.io/inject': 'false'},
  'creationTimestamp': '2022-10-07T01:16:03Z',
  'generation': 1,
  'managedFields': [{'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:annotations': {'.': {},
       'f:sidecar.istio.io/inject': {}}},
     'f:spec': {'.': {},
      'f:predictor': {'.': {},
       'f:sklearn': {'.': {},
        'f:image': {},
        'f:name': {},
        'f:storageUri': {}}}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2022-10-07T01:16:00Z'}],
  'name': 'sklearn-iris',
  'namespace': 'kubeflow-user-example-com',
  'resourceVersion': '17161394',
  'uid': '0ac8cab1-56a7-458d-a2be-30c0778d792e'},
 'spec': {'predictor': {'sklearn': {'image': 'kserve/sklearnserver:v0.9.0',
    'name': 'kserve-container',
    'protocolVersion': 'v1',
    'resources': {'limits': {'cpu': '1', 'mem

In [21]:
KServe.get(service_name, namespace=namespace, watch=True, timeout_seconds=120)
isvc_resp=KServe.get(service_name, namespace=namespace)

NAME                 READY                           PREV                    LATEST URL                                                              
sklearn-iris         Unknown                            0                       100                                                                  
sklearn-iris         False                              0                       100                                                                  
sklearn-iris         False                              0                       100                                                                  
sklearn-iris         False                              0                       100                                                                  
sklearn-iris         Unknown                            0                       100                                                                  
sklearn-iris         Unknown                            0                       100                 

In [22]:
isvc_resp

{'apiVersion': 'serving.kserve.io/v1beta1',
 'kind': 'InferenceService',
 'metadata': {'annotations': {'sidecar.istio.io/inject': 'false'},
  'creationTimestamp': '2022-10-07T01:16:03Z',
  'finalizers': ['inferenceservice.finalizers'],
  'generation': 1,
  'managedFields': [{'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:annotations': {'.': {},
       'f:sidecar.istio.io/inject': {}}},
     'f:spec': {'.': {},
      'f:predictor': {'.': {},
       'f:sklearn': {'.': {},
        'f:image': {},
        'f:name': {},
        'f:storageUri': {}}}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2022-10-07T01:16:00Z'},
   {'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:finalizers': {}},
     'f:spec': {'f:predictor': {'f:containers': {},
       'f:sklearn': {'f:args': {}}}},
     'f:status': {'.': {},
      'f:address': {'.': {}, 'f:url': {}

In [26]:
sklearn_iris_input={
    "instances": [
      [6.8,  2.8,  4.8,  1.4],
      [6.0,  3.4,  4.5,  1.6]
    ]
}

In [27]:
import requests
isvc_url = isvc_resp['status']['address']['url']
print(isvc_url)

response = requests.post(isvc_url, json=sklearn_iris_input)
print(response.text)

http://sklearn-iris.kubeflow-user-example-com.svc.cluster.local/v1/models/sklearn-iris:predict
{"predictions": [1, 1]}


In [ ]:
# curl을 이용한 요청

In [28]:
import requests
import kfp

HOST = "https://192.168.0.30:8080/"
USERNAME = "user@example.com"
PASSWORD = "12341234"

session = requests.Session()
response = session.get(HOST, verify=False) #ssl 접속시 verify=False 

headers = {
    "Content-Type": "application/x-www-form-urlencoded",
}

data = {"login": USERNAME, "password": PASSWORD}
session.post(response.url, headers=headers, data=data)
# session_cookie = session.cookies.get_dict()["authservice_session"]
session_cookie = session.cookies.get_dict()
print(session_cookie)

{'authservice_session': 'MTY2NTEwNTQxNXxOd3dBTkZsSlFsRk5RbE5YVjFNM1ExUlZUMHRIUkVKU1IweFdSMVJCUlZOUU5ESlJSazlHVTFOTU0wNU9ORkZSVDFSWldFOUdNMUU9fIf82wgh4haAOzk3RWOezrFxwLobd44hIi5mfkQBE2TZ'}


In [29]:
import json
headers = {"Host": "sklearn-iris.kubeflow-user-example-com.example.com"}
res = session.post("https://192.168.0.30:8080/v1/models/sklearn-iris:predict", headers=headers, cookies=session_cookie, data=json.dumps(sklearn_iris_input))
print(res.json())

{'predictions': [1, 1]}
